# Upload Text File and download JSON file as a result using nlp.ailab.lv for processing

In [11]:
# set up libraries
import requests
import json
import os
from datetime import datetime
import time

from google.colab import files
import ipywidgets as widgets
from ipywidgets import Button, Layout
from traitlets import link
#global conf


def getNLP(url = "http://nlp.ailab.lv/api/nlp", data = \
           { "data" : { "text":"change me"},\
            "steps":  ["tokenizer", "morpho", "parser", "ner"],\
            "model": "default",\
            "config": None},\
           headers = {'content-type': 'application/json'}):
    """Give a Python object and returns a Python object decoded from JSON"""
#     print(json.dumps(data))
    response = requests.post(url, json.dumps(data), headers=headers)
    if (response.status_code != 200):        
        print(f"Bad response code: {response.status_code}")
        return None
    return response.json()

def getData(text, offset=0,limit=None, steps=["tokenizer", "morpho", "parser", "ner"]):
    if limit:
        txt = text[offset:offset+limit]
    else:
        txt = text[offset:]
    return { "data" : { "text":txt},\
            "steps":  steps,\
            "model": "default",\
            "config": None}

def onDownButtonClick(b):
  global fname # ugly but quick and dirty
  global start
  global limit
  global text

  # print("Got text of len", len(text))
  print(f"{datetime.now().time()} - Sent Request to ailab for processing")
  myjson = getNLP(data=getData(text,start.value, limit.value))
  print(f"{datetime.now().time()} - Received Response from ailab with results")
  jname = fname[:-4] + ".json"
  with open(jname, mode="w", encoding="utf-8") as f:
    json.dump(myjson, f, indent=2)
  print(f"Waiting a few seconds for google to finish writing")
  time.sleep(3)
  files.download(os.path.join(jname))

def handle_text_change(change):
  txtPreview.value=f"{text[start.value:start.value+50]}..."
  endPreview.value=f"...{text[start.value+limit.value-50:start.value+limit.value]}"

def handle_file_change(change):
  global text
  global fname
  fname = fChooser.value
  with open(os.path.join(fname), encoding="utf-8") as f:
    text = f.read()
  handle_text_change(None)

uploaded = files.upload()
fnames = [f for f in uploaded.keys()]
fname = fnames[0] # TODO add check if upload failed
with open(os.path.join(fname), encoding="utf-8") as f:
  text = f.read()

fnames = [f for f in os.listdir() if f.endswith(".txt")]

fChooser = widgets.Dropdown(
    options=fnames,
    value=fname,
    description='Available Text Files',
    disabled=False,
)  
start = widgets.IntText(min=0, max=2_000_000, value=0, description="Start pos")
txtPreview = widgets.Label(value=f"{text[start.value:start.value+50]}...")
startBox = widgets.HBox([start, txtPreview])
start.observe(handle_text_change, names='value')
limit = widgets.IntText(min=0, max=1_000_000, value=1_000, description="Max symbols")
endPreview = widgets.Label(value=f"...{text[start.value+limit.value-50:start.value+limit.value]}")
endBox = widgets.HBox([limit, endPreview])
limit.observe(handle_text_change, names='value')
fChooser.observe(handle_file_change, names='value')
dwnBtn = widgets.Button(description="Process and Download", layout=Layout(width='200px'))
output = widgets.Output()    
display(fChooser, startBox,endBox,dwnBtn)    
dwnBtn.on_click(onDownButtonClick)    

Saving history.txt to history (1).txt
Saving hist.txt to hist (1).txt
Saving h29.txt to h29 (1).txt


Dropdown(description='Available Text Files', index=17, options=('Saucēja balss_1973_417064_504344.txt', 'Būris…

Button(description='Process and Download', layout=Layout(width='200px'), style=ButtonStyle())

11:00:46.200734 - Sent Request to ailab for processing
11:00:48.642319 - Received Response from ailab with results
Waiting a few seconds for google to finish writing
